### 论文阅读笔记 1

- **Date**：_2025-09-02 ~_
- **BY**：_KaiWang AHUT CS_
- **Goal**：_Resilient fault-tolerant anti-synchronization for stochastic delayed reaction-diffusion neural networks with semi-Markov jump parameters (Neural Networks, 2020)_


#### 基本概念

- **反应-扩散**：加入空间维度，系统状态不仅随着时间变化，也随着空间变化：常微分方程->偏微分方程。
- **马尔可夫跳变**：不同模式之间切换，且**驻留时间**服从指数分布（无记忆）。
- **半-马尔可夫跳变**：驻留时间可以是其他分布，不一定无记忆。
- **容器控制**：针对**执行器故障**设计的
- **弹性控制**：针对**增益扰动**设计的


#### 引言

- **研究空白**：
  1. 带有**反应扩散项**的**半马尔可夫时滞神经网络**的**反同步**问题。
  2. 在已有的马尔可夫跳变神经网络的控制器设计中，**执行器故障**和**增益扰动**这两个问题没有被同时考虑过。
- **主要贡献**：
  1. 更加通用的**主从 DRDNNs 模型**
  2. <u>**均方指数稳定条件**</u>
  3. <u>弹性容错控制器的设计策略</u>


#### 预备知识

##### 主系统方程

$$\frac{dz_{1}(t,x)}{dt}=\sum_{k=1}^{q}\frac{\partial}{\partial x_{k}}(D_{k}\frac{\partial z_{1}(t,x)}{\partial x_{k}})-A_{\gamma(t)}z_{1}(t,x) +W_{1\gamma(t)}f(z_{1}(t,x))+W_{2\gamma(t)}f(z_{1}(t-\tau(t),x))$$


**详细解释**：

- $z_{1}(t,x)$：表示在$t$时刻、$x$空间位置，主神经网络的状态向量。
- <u>$\sum_{k=1}^{q}\frac{\partial}{\partial x_{k}}(D_{k}\frac{\partial z_{1}(t,x)}{\partial x_{k}})$：**扩散项**，描述了神经元状态在空间中的传播。</u>
- $-A_{\gamma(t)}z_{1}(t,x)$：**自抑制项**，表示神经元会以一定的速率恢复到静息状态。
- $W_{1\gamma(t)}f(z_{1}(t,x))$：**无时滞的连接项**，描述网络中所有神经元（包括自身）对当前神经元的总影响。
  - $$\underbrace{\sum_y{W}\left( x,y \right) f\left( z_1\left( t,y \right) \right)}_{\text{详细操作}} \quad \xrightarrow{\text{定义/约定为}} \quad \underbrace{W_{1\gamma \left( t \right)}f\left( z_1\left( t,x \right) \right)}_{\text{操作的简洁名称}}$$
- $W_{2\gamma(t)}f(z_{1}(t-\tau(t),x))$：**有时滞的连接项**，描述过去某一时刻（t-τ(t)）的神经元状态对当前的影响。


##### 从系统方程

$$
\begin{aligned}
	dz_2\left( t,x \right) &=\left[ \sum_{k=1}^q{\frac{\partial}{\partial x_k}}\left( D_k\frac{\partial z_2\left( t,x \right)}{\partial x_k} \right) -A_{\gamma \left( t \right)}z_2\left( t,x \right) +W_{1\gamma \left( t \right)}f\left( z_2\left( t,x \right) \right) +W_{2\gamma \left( t \right)}f\left( z_2\left( t-\tau \left( t \right) ,x \right) \right) +B_{\gamma \left( t \right)}u_{\gamma \left( t \right)}^{f}\left( t,x \right) \right] dt\\
\end{aligned}
$$

$$
+\sigma \left( t,z_1\left( t,x \right) +z_2\left( t,x \right) ,z_1\left( t-\tau \left( t \right) ,x \right) +z_2\left( t-\tau \left( t \right) ,x \right) \right) d\omega \left( t \right)
$$


**详细解释**：

- $B_{\gamma \left( t \right)}u_{\gamma \left( t \right)}^{f}\left( t,x \right)$：**控制输入项**，设计一个合适的$u$，让从系统去反同步追踪主系统。
- $\sigma \left( t,z_1\left( t,x \right) +z_2\left( t,x \right) ,z_1\left( t-\tau \left( t \right) ,x \right) +z_2\left( t-\tau \left( t \right) ,x \right) \right) d\omega \left( t \right)$：
  - $dω(t)$：**维纳过程**或**布朗运动**的增量：系统内部的随机波动。
  - $\sigma \left( t,z_1\left( t,x \right) +z_2\left( t,x \right) ,z_1\left( t-\tau \left( t \right) ,x \right) +z_2\left( t-\tau \left( t \right) ,x \right) \right)$：噪声强度函数，决定了扰动的强度，这个强度不是固定的，而是取决于：
    - t：时间。
    - $z_1(t,x) + z_2(t,x)$：系统当前的总状态。
    - $z_1(t-τ(t),x) + z_2(t-τ(t),x)$：系统过去的总状态。


##### 误差系统方程

误差：$\zeta(t,x) = z_{1}(t,x) + z_{2}(t,x)$

误差系统方程：

$$
d\zeta \left( t,x \right) =\left[ \sum_{k=1}^l{\frac{\partial}{\partial x_k}}\left( D_k\frac{\partial \zeta \left( t,x \right)}{\partial x_k} \right) -\left( A_{\gamma \left( t \right)}-B_{\gamma \left( t \right)}\mathcal{G}_{\gamma \left( t \right)}\times \bigl( K_{\gamma \left( t \right)}+\Delta K_{\gamma \left( t \right)}\left( t \right) \bigr) \right) \zeta \left( t,x \right) +W_{1\gamma \left( t \right)}g\left( \zeta \left( t,x \right) \right) +W_{2\gamma \left( t \right)}g\left( \zeta \left( t-\tau \left( t \right) ,x \right) \right) \right] dt
$$

$$
+\sigma \bigl( t,\zeta \left( t,x \right) ,\zeta \left( t-\tau \left( t \right) ,x \right) \bigr) \,d\omega \left( t \right)
$$


- **推导过程**：

  $$
  d\zeta \left( t,x \right) =dz_1\left( t,x \right) +dz_2\left( t,x \right)
  $$

  $$
  =\left[ \sum_{k=1}^q{\frac{\partial}{\partial x_k}}\left( D_k\frac{\partial z_1\left( t,x \right)}{\partial x_k} \right) -A_{\gamma \left( t \right)}z_1\left( t,x \right) +W_{1\gamma \left( t \right)}f\left( z_1\left( t,x \right) \right) +W_{2\gamma \left( t \right)}f\left( z_1\left( t-\tau \left( t \right) ,x \right) \right) \right] dt
  $$

  $$
  +\left[ \sum_{k=1}^q{\frac{\partial}{\partial x_k}}\left( D_k\frac{\partial z_2\left( t,x \right)}{\partial x_k} \right) -A_{\gamma \left( t \right)}z_2\left( t,x \right) +W_{1\gamma \left( t \right)}f\left( z_2\left( t,x \right) \right) +W_{2\gamma \left( t \right)}f\left( z_2\left( t-\tau \left( t \right) ,x \right) \right) +B_{\gamma \left( t \right)}u_{\gamma \left( t \right)}^{f}\left( t,x \right) \right] dt
  $$

  $$
  +\sigma \left( t,z_1\left( t,x \right) +z_2\left( t,x \right) ,z_1\left( t-\tau \left( t \right) ,x \right) +z_2\left( t-\tau \left( t \right) ,x \right) \right) d\omega \left( t \right)
  $$

  令$g(\zeta(t,x)) = f(z_2(t,x)) + f(z_1(t,x))$：

  $$
  d\zeta \left( t,x \right) =\left[ \sum_{k=1}^q{\frac{\partial}{\partial x_k}}\left( D_k\frac{\partial \zeta \left( t,x \right)}{\partial x_k} \right) -A_{\gamma \left( t \right)}\zeta \left( t,x \right) +W_{1\gamma \left( t \right)}g\left( \zeta \left( t,x \right) \right) +W_{2\gamma \left( t \right)}g\left( \zeta \left( t-\tau \left( t \right) ,x \right) \right) +B_{\gamma \left( t \right)}u_{\gamma \left( t \right)}^{f}\left( t,x \right) \right] dt
  $$

  $$
  +\sigma \left( t,\zeta \left( t,x \right) ,\zeta \left( t-\tau \left( t \right) ,x \right) \right) d\omega \left( t \right)
  $$

  $$
  +\sigma \left( t,\zeta \left( t,x \right) ,\zeta \left( t-\tau \left( t \right) ,x \right) \right) d\omega \left( t \right)
  $$

  控制器$u_{\gamma(t)}(t,x) = (K_{\gamma(t)}+\Delta K_{\gamma(t)}(t))\zeta(t,x)$，执行器故障模型为
  $u^f_{\gamma(t)} = \mathcal{G}_{\gamma(t)} u_{\gamma(t)}$，所以$u^f_{\gamma(t)} = \mathcal{G}_{\gamma(t)}(K_{\gamma(t)}+\Delta K_{\gamma(t)}(t))\zeta(t,x)$。
  于是：

  $$
  d\zeta \left( t,x \right) =\left[ \sum_{k=1}^q{\frac{\partial}{\partial x_k}}\left( D_k\frac{\partial \zeta \left( t,x \right)}{\partial x_k} \right) -\left( A_{\gamma \left( t \right)}-B_{\gamma \left( t \right)}\mathcal{G}_{\gamma \left( t \right)}\left( K_{\gamma \left( t \right)}+\Delta K_{\gamma \left( t \right)}\left( t \right) \right) \right) \zeta \left( t,x \right) +W_{1\gamma \left( t \right)}g\left( \zeta \left( t,x \right) \right) +W_{2\gamma \left( t \right)}g\left( \zeta \left( t-\tau \left( t \right) ,x \right) \right) \right] dt
  $$

  $$
  +\sigma \left( t,\zeta \left( t,x \right) ,\zeta \left( t-\tau \left( t \right) ,x \right) \right) d\omega \left( t \right)
  $$


主从系统和被认为是均方指数反同步的，当且仅当误差系统是均方指数稳定的，即：
$$\mathcal{E}\{\int_{\Omega}|\zeta(t,x)|^{2}dx\}\le He^{-\kappa t}$$


#### 系统稳定性分析

#### 控制器设计

#### 数值例子